# Записи телеметрии


|               |    |
|:--------------|:---|
| Группа        | APT29 |
| Описание      | Группа угроз APT29 оперирует как минимум с 2008 года, в 2015 году провела одну из самых известных своих атак. Её содержание ниже. |
| Авторы         | [Воркшоп МЦА](http://mca.nsu.ru/projects/) |
    

## Импортируем библиотеки

In [1]:
from pyspark.sql import SparkSession

## Запускаем сессию Spark

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

## Распаковываем логи атаки

In [3]:
!wget https://github.com/hunters-forge/mordor/raw/master/datasets/large/apt29/day1/apt29_evals_day1_manual.zip

--2020-07-11 10:59:33--  https://github.com/hunters-forge/mordor/raw/master/datasets/large/apt29/day1/apt29_evals_day1_manual.zip
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hunters-forge/mordor/master/datasets/large/apt29/day1/apt29_evals_day1_manual.zip [following]
--2020-07-11 10:59:33--  https://raw.githubusercontent.com/hunters-forge/mordor/master/datasets/large/apt29/day1/apt29_evals_day1_manual.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13944972 (13M) [application/zip]
Saving to: ‘apt29_evals_day1_manual.zip’

apt29_evals_day1_ma 100%[===================>]  13.30M  7.12MB/s    in 

In [4]:
!unzip apt29_evals_day1_manual.zip

Archive:  apt29_evals_day1_manual.zip
  inflating: apt29_evals_day1_manual_2020-05-01225525.json  


## Импортируем логи атаки

In [5]:
df_day1_host = spark.read.json('apt29_evals_day1_manual_2020-05-01225525.json')

## Создаём временный SQL View

In [6]:
df_day1_host.createTempView('apt29Host')

## У таблицы apt29 множество столбцов, их можно посмотреть

In [14]:
df = spark.sql(
'''
DESCRIBE apt29Host
'''
)
df.show(10,truncate = False, vertical = False)

+-------------+---------+-------+
|col_name     |data_type|comment|
+-------------+---------+-------+
|@timestamp   |string   |null   |
|@version     |string   |null   |
|AccessList   |string   |null   |
|AccessMask   |string   |null   |
|AccessReason |string   |null   |
|AccountName  |string   |null   |
|AccountType  |string   |null   |
|Action       |string   |null   |
|Active       |string   |null   |
|ActiveProfile|string   |null   |
+-------------+---------+-------+
only showing top 10 rows



## Нападающий - шаги обнаружения

## 1.A.1. User Execution
**Procedure:** Пользователь исполнил rcs.3aka3.doc

**Criteria:** Процесс rcs.3aka3.doc запущен процессом explorer.exe


### Как правило, данные лежат в логах sysmon. Про поля sysmon можно [почитать здесь](https://docs.microsoft.com/en-us/sysinternals/downloads/sysmon).

In [7]:
df = spark.sql(
'''
SELECT Message
FROM apt29Host
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND LOWER(ParentImage) LIKE "%explorer.exe"
    AND LOWER(Image) LIKE "%3aka3%"

'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Message | Process Create:
RuleName: -
UtcTime: 2020-05-02 02:55:56.157
ProcessGuid: {47ab858c-e13c-5eac-a903-000000000400}

### Security-логи Windows немного не такие, как логи sysmon, иногда информация только в одном из этих логов. См [Sysmon Config.xml recommendations compared to Windows logs](https://static1.squarespace.com/static/552092d5e4b0661088167e5c/t/5d5588b51fd81f0001471db4/1565886646582/Windows+Sysmon+Logging+Cheat+Sheet_Aug_2019.pdf)

In [8]:
df = spark.sql(
'''
SELECT Message
FROM apt29Host
WHERE LOWER(Channel) = "security"
    AND EventID = 4688
    AND LOWER(ParentProcessName) LIKE "%explorer.exe"
    AND LOWER(NewProcessName) LIKE "%3aka3%"

'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 1.A.2. Masquerading
**Procedure:** Использован символ unicode right-to-left override (RTLO) для обфусцирования имени файла rcs.3aka3.doc (изначально cod.3aka.scr)

**Criteria:** Пристутствие right-to-left override character (U+202E) в процессе rcs.3aka.doc ИЛИ в исходном имени файла (cod.3aka.scr)


## Здесь ещё множество тактик, техник и процедур...

## 9.C.3. File Deletion
**Procedure:** Удалён working.zip (из каталога AppData) на диске используя SDelete

**Criteria:** sdelete64.exe удалил файл \AppData\Roaming\working.zip


In [10]:
df = spark.sql(
'''
SELECT Message
FROM apt29Host j
INNER JOIN (
    SELECT h.ProcessGuid
    FROM apt29Host h
    INNER JOIN (
        SELECT f.ProcessGuid
        FROM apt29Host f
        INNER JOIN (
          SELECT d.ProcessGuid
          FROM apt29Host d
          INNER JOIN (
              SELECT b.ProcessGuid
              FROM apt29Host b
              INNER JOIN (
                SELECT ProcessGuid
                FROM apt29Host
                WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                    AND EventID = 1
                    AND ParentImage LIKE '%services.exe'
              ) a
              ON b.ParentProcessGuid = a.ProcessGuid
              WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND Image LIKE '%python.exe'
          ) c
          ON d.ParentProcessGuid = c.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
              AND EventID = 1
        ) e
        ON f.ParentProcessGuid = e.ProcessGuid
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND Image LIKE '%cmd.exe'
    ) g
    ON h.ParentProcessGuid = g.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND h.EventID = 1
) i
ON j.ProcessGuid = i.ProcessGuid
WHERE j.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND j.EventID = 23

'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Message | File Delete:
RuleName: -
UtcTime: 2020-05-02 03:17:41.353
ProcessGuid: {5aa8ec29-e655-5eac-8303-000000000400}
ProcessId: 6640
User: DMEVALS\pbeesly
Image: C:\Windows\Temp\sdelete64.exe
TargetFilename: C:\Users\pbeesly\Desktop\working.zip
Hashes: SHA1=321A74E1D43B00DF8D3D4A55CE36C5E7A143A5C4,MD5=14C4E1AB76CF430C499E8509CA488F54,SHA256=1125328F7A204931E2E22C5BBE7238E382B35C1456F6961B1EF8566C5EE06863,IMPHASH=00000000000000000000000000000000
IsExe

## 9.C.4. File Deletion
**Procedure:** SDelete удалён с диска, использована соманда del cmd.exe

**Criteria:** cmd.exe удалил файл sdelete64.exe


In [11]:
df = spark.sql(
'''
SELECT h.Message
FROM apt29Host h
INNER JOIN (
    SELECT f.ProcessGuid
    FROM apt29Host f
    INNER JOIN (
      SELECT d.ProcessGuid
      FROM apt29Host d
      INNER JOIN (
          SELECT b.ProcessGuid
          FROM apt29Host b
          INNER JOIN (
            SELECT ProcessGuid
            FROM apt29Host
            WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND EventID = 1
                AND ParentImage LIKE '%services.exe'
          ) a
          ON b.ParentProcessGuid = a.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND Image LIKE '%python.exe'
      ) c
      ON d.ParentProcessGuid = c.ProcessGuid
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
    ) e
    ON f.ParentProcessGuid = e.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
      AND EventID = 1
      AND Image LIKE '%cmd.exe'
) g
ON h.ProcessGuid = g.ProcessGuid
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND h.EventID = 23

'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Message | File Delete:
RuleName: -
UtcTime: 2020-05-02 03:17:52.009
ProcessGuid: {5aa8ec29-e618-5eac-7e03-000000000400}
ProcessId: 8772
User: DMEVALS\pbeesly
Image: C:\windows\system32\cmd.exe
TargetFilename: C:\Windows\Temp\sdelete64.exe
Hashes: SHA1=7BCD946326B67F806B3DB4595EDE9FBDF29D0C36,MD5=2B5CB081721B8BA454713119BE062491,SHA256=FEEC1457836A5F84291215A2A003FCDE674E7E422DF8C4ED6FE5BB3B679CDC87,IMPHASH=342934F7499D0F57D88D4434E41B7BF9
IsExecutable: true
Archived: true 

